In [1]:
import time
import random
import math

In [2]:
pessoas = [("Amanda", "CWB"),
           ("Pedro", "GIG"),
           ("Marcos", "POA"),
           ("Priscila", "FLN"),
           ("Jessica", "CNF"),
           ("Paulo", "GYN"),]

In [3]:
destino = "GRU"

In [4]:
voos = {}

In [5]:
with open("voos.txt", "r") as arq:
    for linha in arq.readlines():
        origem, destino, saida, chegada, preco = linha.split(",")
        voos.setdefault((origem, destino), [])
        voos[(origem, destino)].append((saida, chegada, int(preco)))

In [6]:
# [1,4, 3,2, 7,3, 6,3, 2,4, 5,3 ]

def imprimir_agenda(agenda: list[int]) -> None:
    id_voo = -1
    for i in range(len(agenda)//2):
        nome = pessoas[i][0]
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][agenda[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][agenda[id_voo]]
        print(f"{nome} {origem} {ida[0]} {ida[1]} {ida[2]} {volta[0]} {volta[1]} {volta[2]}\n")

In [7]:
agenda = [1,4, 3,2, 7,3, 6,3, 2,4, 5,3 ]

imprimir_agenda(agenda)

Amanda CWB 8:04 10:11 95 12:08 14:05 142

Pedro GIG 10:30 14:57 290 9:49 13:51 229

Marcos POA 17:08 19:08 262 10:32 13:16 139

Priscila FLN 15:34 18:11 326 11:08 14:38 262

Jessica CNF 9:42 11:32 169 12:08 14:47 231

Paulo GYN 13:37 15:08 250 11:07 13:24 171



In [8]:
def obter_minutos(horaStr: str):
    hora = time.strptime(horaStr, "%H:%M")
    minutos = hora[3] * 60 + hora[4]
    return minutos

In [9]:
def custo(solucao) -> None:
    preco_total = 0
    ultima_chegada = 0
    primeira_partida = obter_minutos("23:59")

    id_voo = -1
    for i in range(len(solucao) // 2):
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][solucao[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][solucao[id_voo]]

        preco_total += ida[2] + volta[2]

        if ultima_chegada < obter_minutos(ida[1]):
            ultima_chegada = obter_minutos(ida[1])

        if primeira_partida > obter_minutos(volta[0]):
            primeira_partida = obter_minutos(volta[0])
    
    total_espera = 0
    id_voo = -1
    for i in range(len(solucao) // 2):
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][solucao[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][solucao[id_voo]]

        total_espera += (ultima_chegada - obter_minutos(ida[1])) + (obter_minutos(volta[0]) - primeira_partida)

    if ultima_chegada > primeira_partida:
        preco_total += 50

    return preco_total + total_espera 

In [10]:
custo(agenda)

4635

### PESQUISA RANDÔMICA

In [11]:
def pesquisa_randomica(dominio, funcao_custo):
    melhor_custo = 9999999999
    melhor_solucao = []
    for i in range(0, 1000):
        solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
        custo = funcao_custo(solucao)
        if custo < melhor_custo:
            melhor_custo = custo
            melhor_solucao = solucao
    return melhor_solucao

In [12]:
dominio = [(0, 9)] * (len(pessoas) * 2)
solucao_randomica = pesquisa_randomica(dominio, custo)
custo_randomica = custo(solucao_randomica)
custo_randomica

3549

In [13]:
imprimir_agenda(solucao_randomica)

Amanda CWB 17:11 18:30 108 10:33 12:03 74

Pedro GIG 13:54 18:02 294 9:49 13:51 229

Marcos POA 13:40 15:38 137 9:58 12:56 249

Priscila FLN 11:28 14:40 248 11:08 14:38 262

Jessica CNF 16:43 19:00 246 9:11 10:42 172

Paulo GYN 12:18 14:56 172 9:31 11:43 210



### Subida da Encosta

In [14]:
def subida_da_encosta(dominio, funcao_custo):
    solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
    while True:
        vizinhos = []
        for i in range(len(dominio)):
            if solucao[i] == dominio[i][0]:
                if solucao[i] != dominio[i][1]:
                    vizinhos.append(solucao[0:i] + [solucao[i] + 1] + solucao[i + 1:])
            if solucao[i] < dominio[i][1]:
                if solucao[i] != dominio[i][0]:
                    vizinhos.append(solucao[0:i] + [solucao[i] - 1] + solucao[i + 1:])
            
        atual = funcao_custo(solucao)
        melhor = atual

        for i in range(len(vizinhos)):
            custo = funcao_custo(vizinhos[i])
            if custo < atual:
                melhor = custo
                solucao = vizinhos[i]

        if melhor == atual:
            break
        
    return solucao

In [15]:
dominio = [(0, 9)] * (len(pessoas) * 2)
solucao_subida_da_encosta = subida_da_encosta(dominio, custo)
custo_subida_da_encosta = custo(solucao_subida_da_encosta)
custo_subida_da_encosta

3039

In [16]:
imprimir_agenda(solucao_subida_da_encosta)

Amanda CWB 8:04 10:11 95 8:23 10:28 149

Pedro GIG 10:30 14:57 290 9:49 13:51 229

Marcos POA 12:08 14:59 149 8:19 11:16 122

Priscila FLN 7:34 9:40 324 8:23 11:07 143

Jessica CNF 12:44 14:17 134 7:50 10:08 164

Paulo GYN 12:18 14:56 172 8:04 10:59 136



### Têmpera Simulada

In [17]:
def tempera_simulada(dominio, funcao_custo, temperatura = 10000.0, resfriamento = 0.95, passo = 1):
    solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]

    while temperatura > 0.1:
        index = random.randint(0, len(dominio) - 1)
        direcao = random.randint(-passo, passo)

        solucao_temp = solucao[:]
        solucao_temp[index] += direcao
        if solucao_temp[index] < dominio[index][0]:
            solucao_temp[index] = dominio[index][0]
        if solucao_temp[index] > dominio[index][1]:
            solucao_temp[index] = dominio[index][1]
        
        custo_solucao = funcao_custo(solucao)
        custo_solucao_temp = funcao_custo(solucao_temp)
        probabilidade = pow(math.e, (-custo_solucao - custo_solucao_temp) / temperatura)

        if (custo_solucao_temp < custo_solucao) or (random.random() < probabilidade):
            solucao = solucao_temp
        
        temperatura = temperatura * resfriamento
    return solucao

In [18]:
dominio = [(0, 9)] * (len(pessoas) * 2)
solucao_tempera_simulada = tempera_simulada(dominio, custo)
custo_da_tempera_simulada = custo(solucao_tempera_simulada )
custo_da_tempera_simulada

4120

In [19]:
imprimir_agenda(solucao_tempera_simulada)

Amanda CWB 15:27 17:18 151 10:33 12:03 74

Pedro GIG 6:12 10:22 230 17:14 20:59 277

Marcos POA 15:23 17:25 232 8:19 11:16 122

Priscila FLN 14:01 17:24 338 8:23 11:07 143

Jessica CNF 14:22 16:32 126 12:08 14:47 231

Paulo GYN 9:15 12:03 99 11:07 13:24 171



### Genetic Algorithm

In [22]:
def mutacao(dominio, passo, solucao):
    index = random.randint(0, len(dominio) - 1)
    mutante = solucao

    if random.random() < 0.5:
        if solucao[index] != dominio[index][0]:
            mutante = solucao[0:index] + [solucao[index] - passo] + solucao[index + 1:]
    else:
        if solucao[index] != dominio[index][1]:
            mutante = solucao[0:index] + [solucao[index] + 1] + solucao[index + 1:]
    
    return mutante

def cruzamento(dominio, solucao1, solucao2):
    index = random.randint(1, len(dominio) - 2)

    return solucao1[0:index] + solucao2[index:]

def genetico(dominio, funcao_custo, tamanho_populacao = 50, passo = 1,
             probalidade_mutacao = 0.2, elitismo = 0.2, numero_geracoes = 100):

    populacao = []

    for i in range(tamanho_populacao):
        solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
        populacao.append(solucao)
    
    numero_elitismo = int(elitismo * numero_geracoes)

    for i in range(tamanho_populacao):
        custos = [(funcao_custo(individuo), individuo) for individuo in populacao]
        custos.sort()
        individuos_ordenados = [individuo for (custo, individuo) in custos]
        populacao = individuos_ordenados[0:numero_elitismo]

        while len(populacao) < tamanho_populacao:
            if random.random() < probalidade_mutacao:
                m = random.randint(0, numero_elitismo)
                populacao.append(mutacao(dominio, passo, individuos_ordenados[m]))
            else:
                c1 = random.randint(0, numero_elitismo)
                c2 = random.randint(0, numero_elitismo)
                populacao.append(cruzamento(dominio, individuos_ordenados[c1], individuos_ordenados[c2]))
    return custos[0][1]

In [31]:
dominio = [(0, 9)] * (len(pessoas) * 2)
solucao_genetico = genetico(dominio, custo)
custo_genetico = custo(solucao_genetico )
custo_genetico

3101